In [210]:
import pandas as pd
import numpy as np
import os
import re

pd.set_option('display.max_rows', 50)

In [211]:
# CALCULATE BUDGETS FOR EACH AREA
# link to source: https://app.clarity.so/rawdao/work/33

month = '2022-03'

init_budget = {}

init_budget['governance'] = 2
init_budget['curation'] = 2.5
init_budget['marketing & pr'] = 2.5
init_budget['community'] = 1.5
init_budget['finance'] = 1.5

perf_tranche = {}

perf_tranche['governance'] =  0.4
perf_tranche['curation'] = 0.4
perf_tranche['marketing & pr'] = 0.4
perf_tranche['community'] = 0.5
perf_tranche['finance'] = 0.4

perf_score = {}

perf_score['governance'] =  5/6
perf_score['curation'] = 1/2
perf_score['marketing & pr'] = 2/3
perf_score['community'] = 3/3
perf_score['finance'] = 1/3


final_budget = {}
total_final_budget = 0
total_init = 0

for area in init_budget:
  final_budget[area] = round(init_budget[area]*(1-perf_tranche[area]) + init_budget[area]*(perf_tranche[area]*perf_score[area]),2)
  total_final_budget = total_final_budget + final_budget[area]
  total_init = total_init + init_budget[area]

print(final_budget)
print("total final budget = " + str(round(total_final_budget,2)) + " out of " + str(total_init))

{'governance': 1.87, 'curation': 2.0, 'marketing & pr': 2.17, 'community': 1.5, 'finance': 1.1}
total final budget = 8.64 out of 10.0


In [212]:
# READ AND CLEAN COORDINAPE DATA
dfs = {}
voting_members = {}
total_give = {}

for file in os.listdir('./coordinape_data'):
  try:
    # read data from coordinape files  
    key = re.search('DAO-(.+?)-', file).group(1).lower()
    # print(key)
    dfs[key] = pd.read_csv('./coordinape_data/'+ file)
    
    # calculate voting stats for each area
    voting_members[key] = int(dfs[key]['sent'].astype(bool).sum(axis=0))
    total_give[key] = int(dfs[key]['received'].sum())

    # calculate compensation per area
    dfs[key] = dfs[key].sort_values(by=['received'], ascending = False) # sort
    dfs[key] = dfs[key][['name','address','received']] # select relevant cols
    dfs[key]['percent_comp'] = dfs[key]['received']/total_give[key] # calculate percentage comp.
    dfs[key]['eth_compensation'] = dfs[key]['percent_comp'] * final_budget[key] # calculate actual ETH comp.
    dfs[key] = dfs[key][dfs[key]['eth_compensation'] > 0] # filter people with 0 compensation
    dfs[key]['area'] = key
    dfs[key]['total_budget_of_area'] = final_budget[key]
  except NameError:
    print(file + ' -> file skipped')
    print(key)
    print(NameError)

# prepare area breakdown for reporting
df_export = pd.concat(dfs)

# merge everything and calculate compensation per member
comp_df = pd.concat(dfs)
comp_df = comp_df[['name','address','eth_compensation']]
comp_df = comp_df.groupby(by=['name','address']).sum().reset_index()
comp_df = comp_df.sort_values(by=['eth_compensation'], ascending = False)

print('# of members who voted per area: ' + str(voting_members))
print('total votes per area: ' + str(total_give))
print("total compensated members: " + str(len(comp_df)))

# of members who voted per area: {'community': 17, 'curation': 17, 'finance': 8, 'governance': 16, 'marketing & pr': 19}
total votes per area: {'community': 1758, 'curation': 1692, 'finance': 755, 'governance': 1545, 'marketing & pr': 2069}
total compensated members: 44


In [213]:
# PRINT VOTES PER MEMBER PER AREA
df_export[['name','total_budget_of_area','received','percent_comp','eth_compensation']]


name  total_budget_of_area  received  \
community      26           claudia#0273                  1.50       190   
               28  ezra | shibboleth.eth                  1.50       186   
               4             JoseCactuss                  1.50       170   
               6             Lapsus#4632                  1.50       161   
               34      matthewbrooks.eth                  1.50       149   
...                                  ...                   ...       ...   
marketing & pr 3                 HugoFaz                  2.17         5   
               27          ednz.eth#4780                  2.17         2   
               31             glory#5406                  2.17         2   
               11       MickeySmith#2765                  2.17         1   
               41       samedixsnft#8110                  2.17         1   

                   percent_comp  eth_compensation  
community      26      0.108077          0.162116  
               28      0.105802          0.158703  
               4       0.096701          0.145051  
               6       0.091581          0.137372  
               34      0.084755          0.127133  
...                         ...               ...  
marketing & pr 3       0.002417          0.005244  
               27      0.000967          0.002098  
               31      0.000967          0.002098  
               11      0.000483          0.001049  
               41      0.000483          0.001049  

[142 rows x 5 columns]

In [214]:
# PRINT ETH COMPENSATION PER MEMBER FOR THIS MONTH
comp_df[['name','eth_compensation']]

,name,eth_compensation
25,claudia#0273,0.906549
32,lucaspon,0.721372
5,Lapsus#4632,0.628526
30,glory#5406,0.452348
33,matthewbrooks.eth,0.444038
27,ezra | shibboleth.eth,0.416187
3,JoseCactuss,0.391096
37,nyquist,0.389158
21,brileigh.eth,0.380678
18,b123#2778,0.372668


In [215]:
# CALCULATE RANKINGS FOR LONG TERM COMP MODEL

df_rankings = df_export.reset_index()

# This is where we get into a discussion of how exactly to normalize the ranking. The approach I've chosen here is to weigh each member's score by the areas budget, 
# which is equivalent to using their ETH compensation as the score measurement. This way, members who are performing in areas with higher budget (deemed more important) 
# will score higher in the rankings.

df_rankings['normalized_score'] = df_rankings['percent_comp'] * df_rankings['total_budget_of_area']/total_final_budget # score weighted by area's budget
df_rankings = df_rankings[['name','address','normalized_score','area']]
df_rankings = df_rankings.groupby(by=['name','address']).sum()
df_rankings = df_rankings.sort_values(by=['normalized_score'], ascending = False).reset_index()
df_rankings.index = df_rankings.index + 1
df_rankings['percentile'] = df_rankings.index/len(df_rankings)

# Q1-22 Award Conditions
conditions = [
    (df_rankings['percentile'] <= 0.1),
    (df_rankings['percentile'] > 0.1) & (df_rankings['percentile'] <= 0.25),
    (df_rankings['percentile'] > 0.25) & (df_rankings['percentile'] <= 0.5),
    (df_rankings['percentile'] > 0.5) & (df_rankings['percentile'] <= 0.8),
    (df_rankings['percentile'] > 0.8)
]

tier = [ 
    'top 10%',
    'top 25%',
    'top 50%',
    'bottom 50%',
    'bottom 20%'
]

tokens_awarded = [
    500,
    200,
    100,
    0,
    0
]

coordinape = [ 
    20,
    10,
    5,
    0,
    -200
]


df_rankings['tier'] = np.select(conditions, tier)
df_rankings['$RAW Awarded'] = np.select(conditions, tokens_awarded)
df_rankings['coordinape_change'] = np.select(conditions, coordinape)


print('total $RAW Awarded = ' + str(df_rankings['$RAW Awarded'].sum()))
df_rankings[['name','normalized_score', 'percentile', 'tier', '$RAW Awarded', 'coordinape_change']]


total $RAW Awarded = 4500


,name,normalized_score,percentile,tier,$RAW Awarded,coordinape_change
1,claudia#0273,0.104925,0.022727,top 10%,500,20
2,lucaspon,0.083492,0.045455,top 10%,500,20
3,Lapsus#4632,0.072746,0.068182,top 10%,500,20
4,glory#5406,0.052355,0.090909,top 10%,500,20
5,matthewbrooks.eth,0.051393,0.113636,top 25%,200,10
6,ezra | shibboleth.eth,0.048170,0.136364,top 25%,200,10
7,JoseCactuss,0.045266,0.159091,top 25%,200,10
8,nyquist,0.045041,0.181818,top 25%,200,10
9,brileigh.eth,0.044060,0.204545,top 25%,200,10
10,b123#2778,0.043133,0.227273,top 25%,200,10


In [216]:
# EXPORT RESULTS TO EXCEL
from pathlib import Path
Path("./outputs").mkdir(parents=True, exist_ok=True)

df_export.to_excel("./outputs/"+ month +" - compensation_per_area.xlsx")
comp_df.to_excel("./outputs/"+ month +" - compensation_by_member.xlsx")
df_rankings.to_excel("./outputs/"+ month +" - LT_comp_rankings.xlsx")